# Import OpenTelemetry Modules for Metrics

In [ ]:
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.metrics import MeterProvider
# from opentelemetry.sdk.trace.export import (
#     ConsoleSpanExporter,
#     BatchSpanProcessor
# )
from opentelemetry.exporter.otlp.proto.grpc.metric_exporter import OTLPMetricExporter
from opentelemetry.sdk.metrics import MeterProvider
from opentelemetry.sdk.metrics.export import PeriodicExportingMetricReader
from opentelemetry.metrics import (
    CallbackOptions,
    Observation,
    get_meter_provider,
    set_meter_provider,
)
from opentelemetry.sdk.metrics._internal.export import (
    AggregationTemporality,
    ConsoleMetricExporter
)
from typing import Iterable
from tqdm.notebook import tqdm
import random, socket, time

# Create a MeterProvider that we can host metrics

In [ ]:
otlp_exporter = OTLPMetricExporter(endpoint="localhost:4317", insecure=True)
console_exporter = ConsoleMetricExporter()

reader = PeriodicExportingMetricReader(exporter=otlp_exporter, export_interval_millis=1000)

provider = MeterProvider(metric_readers=[reader], resource=Resource.create({
    # "service.name": "section03",
    "service.name": __name__,
    # "datadog.host.use_as_metadata": True,
    "deployment.environment": "otel",
    "host.name": socket.gethostname(),
}))

set_meter_provider(provider)


# Create some observables

In [ ]:
def observable_counter_func(options: CallbackOptions) -> Iterable[Observation]:
    yield Observation(1, {})


def observable_up_down_counter_func(
    options: CallbackOptions,
) -> Iterable[Observation]:
    yield Observation(-10, {})


def observable_gauge_func(options: CallbackOptions) -> Iterable[Observation]:
    yield Observation(9, {})

In [ ]:
meter = get_meter_provider().get_meter("getting-started", "0.1.2")

# Counter
counter = meter.create_counter("counter")
counter.add(1)

# Async Counter
observable_counter = meter.create_observable_counter(
    "observable_counter",
    [observable_counter_func],
)

# UpDownCounter
updown_counter = meter.create_up_down_counter("updown_counter")
updown_counter.add(1)
updown_counter.add(-5)

# Async UpDownCounter
observable_updown_counter = meter.create_observable_up_down_counter(
    "observable_updown_counter", [observable_up_down_counter_func]
)

# Histogram
histogram = meter.create_histogram("histogram")
histogram.record(99.9)

# Async Gauge
gauge = meter.create_observable_gauge("gauge", [observable_gauge_func])

In [ ]:
updown_counter.add(35)

In [ ]:
updown_counter.add(-10)

Random increment/decrement the updown counter for the next 10 seconds...

In [ ]:
for n in tqdm(range(10)):
    updown_counter.add(random.randint(-20, 20))
    time.sleep(1)

#### End of Section